<a href="https://colab.research.google.com/github/syedmahmoodiagents/Agents/blob/main/RAGNew2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
!huggingface-cli login

In [ ]:

documents = [
    "Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications.",
    "Docker is a platform for developing, shipping, and running applications inside containers.",
    "Machine learning is a field of artificial intelligence that uses statistical techniques to enable computers to learn from data."
]

In [ ]:
retriever_model = SentenceTransformer('bert-base-nli-mean-tokens')
doc_embeddings = retriever_model.encode(documents, convert_to_tensor=True)

In [ ]:

def retrieve(query, top_k=1):
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, doc_embeddings, top_k=top_k)
    retrieved_text = " ".join(documents[hit['corpus_id']] for hit in hits[0])
    return retrieved_text

In [ ]:

def compose_prompt(query):
    context = retrieve(query, top_k=1)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    return prompt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-small")
model = AutoModelForCausalLM.from_pretrained("gpt2-small")

In [ ]:
query = "What is Kubernetes?"

In [ ]:
prompt = compose_prompt(query)

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:

outputs = model.generate(
    **inputs,
    max_length=150,
    num_return_sequences=1,   # Generate a single response
    do_sample=True,           # Enable sampling for more diverse outputs
    top_k=50,
    top_p=0.9,                # Nucleus sampling: only consider tokens with cumulative probability 0.9
    no_repeat_ngram_size=2,   # Avoid repeating n-grams of size 2
    early_stopping=True
)

In [ ]:

response = tokenizer.decode(outputs[0], skip_special_tokens=True)


print("Prompt:\n", prompt)
print("\nResponse:\n", response)